In [33]:
import pandas as pd
import getpass
from sqlalchemy import create_engine
from sqlalchemy.exc import ResourceClosedError
import MySQLdb 
check = False
import json

In [5]:
class MySQLConnection:
    def __init__(self,user,hostname,db):
        self.user = str(user)
        self.hostname = str(hostname)
        self.db = str(db)
        pword = getpass.getpass("Enter password for user {}".format(user))
        # self.cnx = mysql.connector.connect(host=self.hostname,user=self.user, password=pword,database=self.db)
        self.engine = create_engine("mysql+mysqlconnector://{}:{}@{}/{}".format(user,pword,hostname,db))
        self.cnx = self.engine.connect()
    
    def write_to_db(self, df, table_name):
        try:
            df.to_sql(table_name,con = self.cnx,if_exists='fail',index=False)
        except Exception as e:
            print("\n SQL Write error with: ")
            print(df,"\n", e)
    
    def query(self,q):
        try:
            res = pd.read_sql(q,self.cnx)
            return res
        except ResourceClosedError:
            print("Query success, nothing was returned")

In [8]:
sql = MySQLConnection('ubuntu','localhost','classicmodels')

Enter password for user ubuntu········


In [9]:
# https://www.mysqltutorial.org/mysql-sample-database.aspx

In [10]:
sql.query("show tables")

,Tables_in_classicmodels
0,customers
1,employees
2,offices
3,orderdetails
4,orders
5,payments
6,productlines
7,products


In [48]:
# sql.query("create table orderdetails_w_income as select *, priceeach*quantityordered as income from orderdetails")

In [13]:
ienakumi = """
select 
    sum(income) as kopejie_ienakumi,
    city
from 
    customers join 
    (
    select
        orders.customernumber,
        orderdetails_w_income.income
    from 
        orderdetails_w_income join orders on
        orderdetails_w_income.ordernumber = orders.ordernumber
    ) pasutijumu_ienakumi on 
    customers.customernumber = pasutijumu_ienakumi.customernumber
group by
    city
order by 
    kopejie_ienakumi desc
"""

sql.query(ienakumi)

,kopejie_ienakumi,city
0,979880.77,Madrid
1,591827.34,San Rafael
2,497941.50,NYC
3,386514.63,Auckland
4,263997.78,Singapore
...,...,...
72,42570.37,Brisbane
73,41506.19,Los Angeles
74,31310.09,Munich
75,29217.18,Charleroi


In [17]:
sql.query("SELECT status, count(*) as statusi from orders group by status")

,status,statusi
0,Cancelled,6
1,Disputed,3
2,In Process,6
3,On Hold,4
4,Resolved,4
5,Shipped,303


In [41]:
# Uzdevums:
# Sastādiet klientu sarakstu ar kuriem būtu jāsazinās, lai atjaunotu kādu iztrūkstošu personīgo informāciju

In [ ]:
# Uzdevums: 
# 1. Atlasiet kopējos ienākumus no visiem pasūtījumiem, 
# 2. Atlasiet kopējos ienākumus tikai no pasūtījumiem, kas ir piegādāti (shipped)

In [ ]:
# Uzdevums:
# 1. Atlasiet 10 visvairāk pārdotās preces
# 2. Izpētiet datus un izdomājiet pie kādiem nosacījumiem preču skaits būtu jāpapildina 
# (pieņemsim, ka pārdošanai pieejamo preču skaits ir papildināms par jebkādu skaitu 2 dienās)
# 3. Atlasiet preces, kuras būtu jāiepērk pēc šiem izvēlētajiem kritērijiem

In [ ]:
# Uzdevums:
# Uzrakstiet SQL vaicājumu, kurš saskaita cik ir darbinieku, kuriem ir padotie

In [47]:
# Uzdevums:
# Uzrakstiet SQL vaicājumu, kurš katram customer id piekārto visnesenāko maksājumu

# Katram customer var būt vairāki pirkumi.
sql.query("select customernumber, count(*) as pirkumi from payments group by customernumber order by pirkumi desc")

,customernumber,pirkumi
0,141,13
1,124,9
2,145,4
3,398,4
4,148,4
...,...,...
93,473,2
94,415,1
95,211,1
96,450,1


In [21]:
# Uzdevums: 
# Uzrakstiet python funkciju, kuras input ir valīds produkta kods un output ir json teksta pavediens, 
# kas satur lauka nosaukumu un informāciju par preci.
# Piem. ja funkcijas input būtu S700_3505, 
# tad funkcijai būtu jāatgriež sekojošs teksta pavediens (piemērā sadalīts vairākās rindiņās lasāmībai):


# {
# "product_description":"Completed model measures 19 1\\/2 inches long, 9 inches high, 3inches 
# wide and is in barn red\\/black. All wood and metal.",
# "product_name":"The Titanic",
# "original_price": 51.09,
# "quantity_in_stock":1956
# }


In [ ]:
# Uzdevums:
# Papildiniet šo funkciju ar funkcionalitāti, kas json pavedienā, string mainīgo vietā atgriež "default" un skaitļu vietā 0, 
# ja nav atrasts neviens produkts ar šādu produkta kodu